In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import threading

# Define the TuringMachine and TwoTapeTuringMachine classes here
class TuringMachine:
    def __init__(self, states, symbols, blank_symbol, transitions, initial_state, accepting_states):
        self.states = states
        self.symbols = symbols
        self.blank_symbol = blank_symbol
        self.transitions = transitions
        self.state = initial_state
        self.accepting_states = accepting_states
        self.tape = [blank_symbol]
        self.head_position = 0
        
    def step(self):
        current_symbol = self.tape[self.head_position]
        if (self.state, current_symbol) in self.transitions:
            new_state, new_symbol, move = self.transitions[(self.state, current_symbol)]
            self.state = new_state
            self.tape[self.head_position] = new_symbol
            if move == 'R':
                self.head_position += 1
                if self.head_position == len(self.tape):
                    self.tape.append(self.blank_symbol)
            elif move == 'L':
                self.head_position -= 1
                if self.head_position < 0:
                    self.tape.insert(0, self.blank_symbol)
                    self.head_position = 0
            return True
        return False
    
    def run(self):
        while self.state not in self.accepting_states:
            if not self.step():
                break
        return self.tape

class TwoTapeTuringMachine(TuringMachine):
    def __init__(self, *args):
        super().__init__(*args)
        self.tape2 = [self.blank_symbol]
        self.head_position2 = 0
        
    def step(self):
        current_symbol = self.tape[self.head_position]
        current_symbol2 = self.tape2[self.head_position2]
        if (self.state, current_symbol, current_symbol2) in self.transitions:
            new_state, new_symbol, new_symbol2, move, move2 = self.transitions[(self.state, current_symbol, current_symbol2)]
            self.state = new_state
            self.tape[self.head_position] = new_symbol
            self.tape2[self.head_position2] = new_symbol2
            if move == 'R':
                self.head_position += 1
                if self.head_position == len(self.tape):
                    self.tape.append(self.blank_symbol)
            elif move == 'L':
                self.head_position -= 1
                if self.head_position < 0:
                    self.tape.insert(0, self.blank_symbol)
                    self.head_position = 0
            if move2 == 'R':
                self.head_position2 += 1
                if self.head_position2 == len(self.tape2):
                    self.tape2.append(self.blank_symbol)
            elif move2 == 'L':
                self.head_position2 -= 1
                if self.head_position2 < 0:
                    self.tape2.insert(0, self.blank_symbol)
                    self.head_position2 = 0
            return True
        return False


# Define the user input functions here
def get_input():
    machine_type = input("Enter '1' for one-tape Turing machine, '2' for two-tape Turing machine: ")
    states = input("Enter states separated by space: ").split()
    symbols = input("Enter symbols separated by space: ").split()
    blank_symbol = input("Enter the blank symbol: ")
    initial_state = input("Enter the initial state: ")
    accepting_states = input("Enter accepting states separated by space: ").split()
    transitions = get_transitions(machine_type)
    initial_tape = list(input("Enter initial tape content: "))
    
    return {
        "machine_type": machine_type,
        "states": states,
        "symbols": symbols,
        "blank_symbol": blank_symbol,
        "transitions": transitions,
        "initial_state": initial_state,
        "accepting_states": accepting_states,
        "initial_tape": initial_tape
    }

def get_transitions(machine_type):
    transitions = {}
    while True:
        transition = input("Enter transition (or 'done' to finish): ")
        if transition == 'done':
            break
        transition_parts = transition.split()
        if machine_type == '1':
            if len(transition_parts) != 5:
                print("Invalid transition! Please enter transitions in the format: current_state read_symbol new_state write_symbol move_direction")
                continue
            current_state, read_symbol, new_state, write_symbol, move_direction = transition_parts
            transitions[(current_state, read_symbol)] = (new_state, write_symbol, move_direction)
        elif machine_type == '2':
            if len(transition_parts) != 7:
                print("Invalid transition! Please enter transitions in the format: current_state read_symbol1 read_symbol2 new_state write_symbol1 write_symbol2 move_direction1 move_direction2")
                continue
            current_state, read_symbol1, read_symbol2, new_state, write_symbol1, write_symbol2, move_direction1, move_direction2 = transition_parts
            transitions[(current_state, read_symbol1, read_symbol2)] = (new_state, write_symbol1, write_symbol2, move_direction1, move_direction2)
    return transitions


# Define the simulation and visualization functions here
import time
import sys

def visualize_turing_machine(tm):
    def print_tape(tape, head_position):
        # Remove trailing blank symbols for display
        while tape and tape[-1] == tm.blank_symbol:
            tape.pop()
        tape_str = ' '.join(tape)
        head_indicator = ' ' * (2 * head_position) + '^'
        display(HTML(f"<pre>{tape_str}\n{head_indicator}</pre>"))
    
    display(HTML("<b>Initial Configuration:</b>"))
    print_tape(tm.tape.copy(), tm.head_position)
    display(HTML(f"<b>Initial State:</b> {tm.state}"))
    display(HTML("<b>Running...</b>"))
    time.sleep(1)
    
    iteration = 0
    while tm.state not in tm.accepting_states:
        if not tm.step():
            display(HTML("<b>Halting...</b>"))
            break
        iteration += 1
        display(HTML(f"<b>Iteration {iteration}:</b>"))
        print_tape(tm.tape.copy(), tm.head_position)
        display(HTML(f"<b>Current State:</b> {tm.state}"))
        time.sleep(1)
    
    display(HTML("<b>Final Configuration:</b>"))
    print_tape(tm.tape.copy(), tm.head_position)
    display(HTML(f"<b>Final State:</b> {tm.state}"))
    display(HTML(f"<b>Head Position:</b> {tm.head_position}"))

def run_turing_machine(config):
    if config["machine_type"] == '1':
        tm = TuringMachine(config["states"], config["symbols"], config["blank_symbol"], config["transitions"], config["initial_state"], config["accepting_states"])
    elif config["machine_type"] == '2':
        tm = TwoTapeTuringMachine(config["states"], config["symbols"], config["blank_symbol"], config["transitions"], config["initial_state"], config["accepting_states"])
    
    tm.tape = config["initial_tape"] + [config["blank_symbol"]]
    visualize_turing_machine(tm)

# Frontend components
machine_type_dropdown = widgets.Dropdown(options=['1', '2'], description='Machine Type:')
states_text = widgets.Text(description='States:')
symbols_text = widgets.Text(description='Symbols:')
blank_symbol_text = widgets.Text(description='Blank Symbol:')
initial_state_text = widgets.Text(description='Initial State:')
accepting_states_text = widgets.Text(description='Accepting States:')
transitions_textarea = widgets.Textarea(description='Transitions:')
initial_tape_text = widgets.Text(description='Initial Tape:')
run_button = widgets.Button(description='Run')
reset_button = widgets.Button(description='Reset')
output_area = widgets.Output()

# Frontend logic
def run_simulation(b):
    with output_area:
        clear_output(wait=True)
        config = {
            "machine_type": machine_type_dropdown.value,
            "states": states_text.value.split(),
            "symbols": symbols_text.value.split(),
            "blank_symbol": blank_symbol_text.value,
            "transitions": parse_transitions(transitions_textarea.value, machine_type_dropdown.value),
            "initial_state": initial_state_text.value,
            "accepting_states": accepting_states_text.value.split(),
            "initial_tape": list(initial_tape_text.value)
        }
        run_turing_machine(config)

def reset_fields(b):
    machine_type_dropdown.value = '1'
    states_text.value = ''
    symbols_text.value = ''
    blank_symbol_text.value = ''
    initial_state_text.value = ''
    accepting_states_text.value = ''
    transitions_textarea.value = ''
    initial_tape_text.value = ''
    with output_area:
        clear_output(wait=True)

def parse_transitions(transitions_str, machine_type):
    transitions = {}
    for line in transitions_str.split('\n'):
        parts = line.split()
        if not parts:
            continue
        if machine_type == '1':
            if len(parts) == 5:
                transitions[(parts[0], parts[1])] = (parts[2], parts[3], parts[4])
        elif machine_type == '2':
            if len(parts) == 7:
                transitions[(parts[0], parts[1], parts[2])] = (parts[3], parts[4], parts[5], parts[6])
    return transitions

# Event handlers
run_button.on_click(run_simulation)
reset_button.on_click(reset_fields)

# Display frontend
display(machine_type_dropdown, states_text, symbols_text, blank_symbol_text, initial_state_text, accepting_states_text, transitions_textarea, initial_tape_text, run_button, reset_button, output_area)


Dropdown(description='Machine Type:', options=('1', '2'), value='1')

Text(value='', description='States:')

Text(value='', description='Symbols:')

Text(value='', description='Blank Symbol:')

Text(value='', description='Initial State:')

...

Textarea(value='', description='Transitions:')

Text(value='', description='Initial Tape:')

Button(description='Run', style=ButtonStyle())

Button(description='Reset', style=ButtonStyle())

Output()